In [2]:
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile
from ahocorasick import Automaton
from tqdm.auto import tqdm

# Code to clean the data

In [3]:
FILE = "./italian_restaurants.zip"
zip_file = ZipFile(FILE)
csv_file = zip_file.namelist()[0]
data = pd.read_csv(zip_file.open(csv_file))

/var/folders/jb/ntxm50wx2wv90pnq36yd8w940000gn/T/ipykernel_28151/804822752.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(zip_file.open(csv_file))


In [4]:
# data.iloc[0]

Removing useless fields

In [5]:
df = data.drop(["claimed", 
           "awards", 
           "popularity_detailed", 
           "popularity_generic", 
           "price_range", 
           "features", 
           "open_days_per_week", 
           "open_hours_per_week", 
           "working_shifts_per_week",
           "default_language",
           "reviews_count_in_default_language",
           "excellent",
           "very_good",
           "average",
           "poor",
           "terrible",
           "food",
           "service",
           "value",
           "atmosphere",
           "keywords"], axis=1)

Populating the "City" field taking it from the address

In [6]:
comuni_df = pd.read_csv("./elenco_comuni_italiani.csv", encoding='iso-8859-1', delimiter=";")

In [7]:
#TODO: "Rome" non viene preso, quindi risolvi questa cosa
def build_automaton(keywords):
    automaton = Automaton()
    for keyword in keywords:
        automaton.add_word(keyword, keyword)
    automaton.make_automaton()
    return automaton

keywords = [row['Denominazione in italiano'].lower() for _, row in comuni_df.iterrows()]
automaton = build_automaton(keywords)

def find_comune(address):
    found = set()
    for item in automaton.iter(",".join(address.split(",")[1:]).lower()):
        found.add(item[1])
    if found:
        return max(found, key=len)
    else:
        return None

tqdm.pandas()
df['city'] = df['address'].progress_apply(find_comune)



  0%|          | 0/224763 [00:00<?, ?it/s]

In [8]:
final_df = df.copy()
final_df['total_reviews_count'] = final_df['total_reviews_count'].fillna(0.0)
final_df['avg_rating'] = final_df['avg_rating'].fillna(0.0)
final_df = final_df.fillna("")
final_df = final_df[final_df["latitude"] != ""]
final_df = final_df[final_df["longitude"] != ""]

In [9]:
df = final_df.reset_index()

### Save the dataframe as a zipped file

In [10]:
# from zipfile import ZipFile
# import os

# DF_PATH = "./italian_restaurant_clean.csv"
# ZIP_PATH = "./italian_restaurant_clean.zip"
# final_df.to_csv(DF_PATH)
# with ZipFile(ZIP_PATH, "w") as myzip:
#     myzip.write(DF_PATH)

# os.remove(DF_PATH)

# Code to insert the restaurants in the database 

In [11]:
# df = pd.read_csv("./italian_restaurant_clean.zip")

In [12]:
import tensorflow as tf
import tensorflow_hub as hub

# Load the Universal Sentence Encoder
# use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
use_model = hub.load("https://tfhub.dev/tensorflow/tfjs-model/universal-sentence-encoder-lite/1/default/1")
def embed_name(name):
    embeddings = use_model([name])
    return embeddings[0]

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-18 12:22:30.620331: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-18 12:22:30.620378: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-05-18 12:22:40.657687: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-18 12:22:40.712824: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [36]:
restaurant1 = "Mariuccio"
restaurant2 = "L'Ortal"

# Embed the restaurant names
embedding1 = embed_name(restaurant1)
embedding2 = embed_name(restaurant2)

# Compute similarity between the embeddings
distance = 1 - np.inner(embedding1, embedding2)

print(embedding1)
print(distance)

tf.Tensor(
[ 1.64556336e-02  1.15570668e-02 -4.57089692e-02 -3.25988345e-02
 -2.11590715e-02 -2.27180999e-02 -1.67041887e-02 -3.47554274e-02
  1.50391553e-03 -1.33230956e-02 -2.25194395e-02  6.89918026e-02
 -2.49969885e-02  9.22679808e-03 -7.92731065e-03  3.90731124e-03
 -4.97879609e-02  4.63860994e-03  6.85884878e-02  8.86448547e-02
  8.12171549e-02  5.05549721e-02 -1.80041511e-02  1.20137427e-02
  3.82102877e-02 -5.80813028e-02  3.02581321e-02  3.30682397e-02
 -2.58147027e-02  9.90221202e-02  1.03083886e-02  4.06892449e-02
 -5.25266193e-02  5.53741232e-02  1.81524474e-02 -3.12040583e-03
 -9.87769663e-03  7.14933723e-02 -2.73623932e-02 -3.08117047e-02
  1.47487491e-03  6.39161095e-02 -2.17536725e-02 -3.69621404e-02
  7.08925352e-02  2.43635476e-02  2.68492829e-02 -7.92923644e-02
 -1.79155916e-02 -3.63301001e-02  2.98728119e-03  1.00280391e-02
  1.49169173e-02  2.69088289e-03 -1.58063099e-02 -3.53279896e-02
  6.59534782e-02 -3.05400249e-02 -3.03522144e-02 -2.14141086e-02
  4.06397544e-

In [14]:
from minio import Minio

URL = 'http://localhost:3000/restaurants/create-restaurant'

client = Minio(
    "localhost:9002",
    access_key="rxSdZSDLIM5nJcr1",
    secret_key="ATjj0yffTeQqIAgVC7GcxONoNGT0l7hS",
    secure=False  # Otherwise bug
)

In [18]:
import json
import io

def insert_embeddings_batch(batch):
    BUCKET_NAME = "embeddings"
    objects = []
    lengths = []
    object_names = []
    for index, row in tqdm(batch.iterrows(), desc="Processing batch", total=len(batch)):
        embedding = embed_name(row["restaurant_name"]).numpy().tolist()
        embedding_str = json.dumps(embedding)
        embedding_bytes = embedding_str.encode("utf-8")
        objects.append(io.BytesIO(embedding_bytes))
        lengths.append(len(embedding_bytes))
        object_names.append(f"embedding_{row['index']}")

    for obj_name, obj, length in zip(object_names, objects, lengths):
        client.put_object(
            BUCKET_NAME,
            obj_name,
            obj,
            length
        )

BATCH_SIZE = 1000  # Define your batch size

tqdm.pandas()

# Split the dataframe into batches
batches = [df[i:i+BATCH_SIZE] for i in range(0, len(df), BATCH_SIZE)]

# Process each batch
for batch in tqdm(batches):
    insert_embeddings_batch(batch)


  0%|          | 0/223 [00:00<?, ?it/s]

Processing batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Processing batch:   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
def get_num_images():
    objects = client.list_objects(bucket_name="images")
    num_images = sum(1 for _ in objects)
    return num_images

num_images = get_num_images()

def assign_image(index):
    return index % num_images

def create_restaurant(row):
    body = {
            "name": row["restaurant_name"],
            "embeddingName": f"embedding_{(row['index'])}",
            "imageName": f"restaurant_image_{assign_image(row['index'])}",
            "address": row["address"],
            "latitude": float(row["latitude"]),
            "longitude": float(row["longitude"]),
            "country": row["country"],
            "region": row["region"],
            "province": row["province"],
            "city": row["city"],
            "tags": row["top_tags"],
            "cuisines": row["cuisines"],
            "specialDiets": row["special_diets"],
            "priceLevel": row["price_level"],
            "meals": row["meals"],
            "avgRating": float(row["avg_rating"]),
            "vegetarianFriendly": row["vegetarian_friendly"] == "Y",
            "veganFriendly": row["vegan_options"] == "Y",
            "glutenFree": row["gluten_free"] == "Y",
            "reviewsNumber": int(row["total_reviews_count"])
        }

    
    result = requests.post(URL, json=body)

import json
import io
def insert_embeddings(row):
    BUCKET_NAME = "embeddings"
    embedding =  embed_name(row["restaurant_name"]).numpy().tolist()
    embedding_str = json.dumps(embedding)
    embedding_bytes = embedding_str.encode("utf-8")
    client.put_object(
            BUCKET_NAME, f"embedding_{row['index']}", io.BytesIO(embedding_bytes), length=len(embedding_bytes))

tqdm.pandas()
df.progress_apply(create_restaurant, axis=1)
# df.progress_apply(insert_embeddings, axis=1)

  0%|          | 0/222603 [00:00<?, ?it/s]

KeyboardInterrupt: 